# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Part A: A dynamical system on a network

This lab accompanies the slides from class: [PDF slides on T-Square](https://t-square.gatech.edu/access/content/group/gtc-239f-fc11-5690-9dae-2dc96b59f372/cx4230-sp17--25--dyn-sys-nets.pdf)

## A conceptual model of "opinion dynamics"

To illustrate the interesting dynamics possible on a network, we'll look at an example considered by [Pluchino et al. (2005)](http://arxiv.org/abs/cond-mat/0410217v2). This paper asked the following question about a network of interacting agents:

> Can agents with different intitial states converge to one or a few uniform states?

They were especially interested in a social network of people where the "states" are differing opinions: if everyone has a different opinion initally, can they still reach agreement? They were inspired by biological systems, like [synchronizing fireflies](https://www.youtube.com/watch?v=a-Vy7NZTGos). For more fun background on this topic, see this TED talk by Steven Strogatz: https://www.youtube.com/watch?v=aSNrKS-sCE0

In Pluchino et al.'s mathematical model, they considered $x_i \equiv x_i(t) \in \mathcal{R}$ to be some measure of the "opinion" of person $i$. It can be positive or negative, and it can take any value in $(-\infty, +\infty)$. These people are connected in a social network, represented as an undirected graph given by its adjacency matrix, $A \equiv (a_{ij})$, where $a_{ij} = 1$ if persons $i$ and $j$ are connected, and 0 otherwise.

They hypothesized a model of the dynamics of $x_i(t)$ by the system of ODEs,

$$\dfrac{dx_i}{dt} \equiv f_i(\vec{x}) \equiv w_i + \dfrac{K}{d_i} \sum_j a_{ij} \cdot \sin \theta_{ij} \cdot \exp \left( -\alpha \cdot \theta_{ij}^2 \right),$$

where $K$ and $\alpha$ are model parameters, $d_i$ is the degree (number of neighbors) of $i$, and $\theta_{ij} \equiv x_j - x_i$, that is, the difference between the opinions of person $i$ and $j$. This formula expresses several modeling ideas.

One idea is that there is a person-specific rate of change, $w_i$. They took $w_i$ in a population to be randomly distributed; in this notebook, let's assume a normal distribution with some prescribed variance $\sigma^2$, so that $w_i \sim \mathcal{N}(0, \sigma^2)$.

The other ideas are about how the interaction between a pair $(i, j)$ of _connected_ people (i.e., $a_{ij} = 1$), expressed relative to their difference, $\theta_{ij}$.

Consider small positive values of $\theta_{ij}$, meaning person $j$ feels a little more positively than person $i$; then $\sin \theta_{ij}$ will be positive and thus influencing the change in person $i$'s opinion to be more positive. Then, at a certain point ($\theta_{ij} = \pi$), the two differ by enough that they start to polarize ($\sin \theta_{ij} < 0$). These oscillations continue for still larger $\theta_{ij}$. However, the oscillatory behavior is moderated by the difference, as expressed in the $\exp \left( -\alpha \cdot \theta_{ij}^2 \right)$ term: the farther the difference the less the influence. The interactions are summed across all neighbors $j$. Lastly, there is a final normalizing factor consisting of a parameter $K$ and normalization by the number of neighbors, or degree, $d_i$.

Lastly, observe that the $a_{ij}$ in the sum acts as a filter: only $\theta_{ij}$ where $a_{ij} \neq 0$ contribute. Let's use this fact to simplify the notation of the system accordingly. Let

* $\vec{w} \equiv (w_i)$ be the vector of person-specific change rates;
* $D \equiv \mbox{diag}(d_i)$ be the _diagonal_ matrix of degrees;
* $\Theta \equiv (\theta_{ij})$ be the matrix of all pairwise differences;
* $A \odot B$ denote elementwise multiplication of matrices $A$ and $B$; and
* $\vec{u} \equiv (1, ..., 1)^T$ be the vector of all ones.

The the (sparse) matrix of _filtered_ pairwise differences is $\hat{\Theta} \equiv A \odot \Theta$.

Now the entire system of ODEs can be written as,

$$
  \dfrac{d\vec{x}}{dt} = \vec{f}(\vec{x}) = \vec{w} + K D^{-1} \cdot \sin (\hat{\Theta}) \cdot \exp (-\alpha \cdot \hat{\Theta} \odot \hat{\Theta}) \cdot \vec{u}.
$$

## Implementing the model: Preliminaries

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse
from scipy.integrate import odeint
import math

In [ ]:
# Core plotting support
import matplotlib.pyplot as plt
%matplotlib inline

For simplicity, let's assume a 2-D grid graph, that is, the graph that is an $n \times n$ lattice of nodes, where each node is connected to its 8 nearest neighbors. The following code cell downloads a handy module that helps construct the matrix $A$ corresponding to this graph.

In [ ]:
import os ; os.makedirs ('cx4230', exist_ok=True)

# http://stackoverflow.com/questions/7243750/download-file-from-web-in-python-3
import urllib.request
MOD_URL = 'https://raw.githubusercontent.com/rvuduc/cx4230sp17labs/master/lab10/pyamg_stencil.py'
MOD_LOCAL = 'cx4230/pyamg_stencil.py'
urllib.request.urlretrieve (MOD_URL, MOD_LOCAL)

In [ ]:
%reload_ext autoreload
%autoreload 2

from cx4230 import pyamg_stencil
from cx4230.pyamg_stencil import stencil_grid

In [ ]:
# Demo
def gen_grid_graph (n):
    S_2d_8nn = np.array ([[1, 1, 1],
                          [1, 0, 1],
                          [1, 1, 1]])
    return stencil_grid (S_2d_8nn, (n, n)).tocsr ()

n = 4
A_2d = gen_grid_graph (n)

print (A_2d.shape)
print (A_2d.todense ())

Given an arbitrary binary adjacency matrix for an undirected graph, the following function computes the degree of each node.

In [ ]:
def degrees (A):
    return np.squeeze (np.asarray (A.sum (axis=1)))

# Demo
np.reshape (degrees (A_2d), (n, n))

**Exercise 1** (2 points). Write a Python function that, given

* a vector `x[:]` of state values, $\vec{x} \equiv (x_i)$; and
* a matrix `A[:, :]`, which is the adjacency matrix $A \equiv (a_{ij})$ for an undirected graph stored as a [sparse matrix](http://docs.scipy.org/doc/scipy/reference/sparse.html);

write a function that returns the matrix $\hat{\Theta} \equiv (a_{ij} \cdot (x_j - x_i))$ of differences, also stored as a _sparse matrix_.

In [ ]:
def calc_sparse_diff_pairs (x, A):
    # YOUR CODE HERE
    raise NotImplementedError()

# Demo
np.set_printoptions (linewidth=100)
calc_sparse_diff_pairs (np.arange (25), A_2d).todense ()

Given the preceding function, we can implement the right-hand side of the model, $\vec{f} \equiv (f_i(\vec{x}))$:

$$
  \dfrac{d\vec{x}}{dt} = \vec{f}(\vec{x}) = \vec{w} + K D^{-1} \cdot \sin (\hat{\Theta}) \cdot \exp (-\alpha \cdot \hat{\Theta} \odot \hat{\Theta}) \cdot \vec{u}.
$$

In [ ]:
def F (x, A, w, k, alpha):
    d = degrees (A)
    Theta_hat = calc_sparse_diff_pairs (x, A)

    I, J = np.nonzero (A)
    V = Theta_hat[I, J] # non-zero values of Theta_hat
    
    V_S = np.sin (V)
    V_E = np.exp (-alpha * np.multiply (V, V))
    V = np.multiply (V_S, V_E)
    V = np.squeeze (np.asarray (V))
    S = sp.sparse.coo_matrix ((V, (I, J)))
    
    u = np.ones (x.shape)
    
    return w + k/d * (S.dot (u))

In [ ]:
n = 8
SIGMA = 1.0
N = n*n
w = np.random.normal (scale=SIGMA, size=N)
K = 0.1
ALPHA = 1.0
A_2d = gen_grid_graph (n)

x_0 = np.zeros (N)
print ("x_0:", x_0)
print ("F (x_0):", F (x_0, A_2d, w, K, ALPHA))

def F_ode (x, t, A, w, k, alpha):
    return F (x, A, w, k, alpha)

t_all = [0.0, 10.0]
x_t = odeint (F_ode, x_0, t_all, args=(A_2d, w, K, ALPHA))
print ("x_t:", x_t)
plt.pcolor (np.reshape (x_t[1], (n, n)))
plt.colorbar ()
plt.axes().set_aspect('equal')

In [ ]:
n = 8
SIGMA = 1.0
N = n*n
w = np.random.normal (scale=SIGMA, size=N)
print ("n:", n)
print ("w:", w)

def sim (x_0, k, alpha, t):
    t_all = np.array ([0.0, t])
    x_t = odeint (F_ode, x_0, t_all, args=(A_2d, w, k, alpha))
    return x_t[1]

In [ ]:
def isim (k=0.0, alpha=1.0, t=0.0):
    x_t = sim (x_0, k, alpha, t)
    plt.pcolor (np.reshape (x_t, (n, n)))
    plt.colorbar ()
    plt.axes().set_aspect('equal')
    print ("(k=%g, alpha=%g, t=%g) done!" % (k, alpha, t))

In [ ]:
from ipywidgets import interact
interact (isim, k=(0.0, 10.0, 1.0), alpha=(0.1, 1.0, 0.1), t=(0.0, 10.0, 1.0)) ;

**Exercise 2** (2 points). What do you observe as `k` increases from `0` to `10`? Does it depend on `alpha`?

YOUR ANSWER HERE